# Calculate global diagnostics of LFRic THAI runs

In [1]:
import warnings

warnings.filterwarnings("ignore")

In [2]:
from functools import partial
from operator import itemgetter

import cartopy.crs as ccrs
import iris
import matplotlib.colors as mcol
import matplotlib.patheffects as PathEffects
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from iris.experimental import stratify
from iris.analysis.maths import apply_ufunc
from matplotlib.offsetbox import AnchoredText

In [3]:
from tqdm.notebook import tqdm

In [4]:
import aeolus
from aeolus.calc import (
    after_n_day_mean,
    between_day_mean,
    last_n_day_mean,
    sigma_p,
    spatial_mean,
    time_mean,
    toa_cloud_radiative_effect,
    bond_albedo,
    ghe_norm,
    zonal_mean,
)
from aeolus.const import init_const
from aeolus.coord import (
    get_cube_rel_days,
    get_xy_coords,
    interp_cube_from_height_to_pressure_levels,
    interp_cubelist_from_height_to_pressure_levels,
    isel,
    regrid_3d,
    replace_z_coord,
    roll_cube_0_360,
    roll_cube_pm180,
)
from aeolus.core import AtmoSim
from aeolus.io import create_dummy_cube, load_vert_lev
from aeolus.meta import const_from_attrs, update_metadata
from aeolus.model import um
from aeolus.plot import (
    add_custom_legend,
    cube_minmeanmax_str,
    subplot_label_generator,
    tex2cf_units,
    unit_format,
)
from aeolus.subset import (
    extract_after_n_days,
    extract_between_days,
    extract_last_n_days,
    unique_cubes,
)

In [5]:
import pouch
from pouch.clim_diag import calc_derived_cubes
from pouch.plot import (
    KW_AUX_TTL,
    KW_CART,
    KW_MAIN_TTL,
    KW_SBPLT_LABEL,
    KW_SYM0,
    KW_ZERO_LINE,
    figsave,
    linspace_pm1,
    use_style,
)

KW_SYM0 = {**KW_SYM0, "cmap": "seismic"}

import paths
import quickplot as qplt
from lfric_model import lfric
from lfric_util import load_lfric_proc

In [6]:
plt.style.use("custom.mplstyle")
# plt.style.use("seaborn-v0_8-darkgrid")
pouch.RUNTIME.figsave_stamp = False

In [7]:
import scooby

scooby.doo(
    core=sorted(["aeolus", "iris", "matplotlib", "numpy", "dask"]),
    optional=[],
)

--------------------------------------------------------------------------------
  Date: Wed Jan 25 16:37:18 2023 UTC

                OS : Linux
            CPU(s) : 192
           Machine : x86_64
      Architecture : 64bit
               RAM : 503.5 GiB
       Environment : Jupyter
       File system : ext4

  Python 3.10.8 | packaged by conda-forge | (main, Nov 22 2022, 08:26:04) [GCC
  10.4.0]

            aeolus : 0.4.16+4.g30f9555
              dask : 2022.11.0
              iris : 3.4.0
        matplotlib : 3.6.2
             numpy : 1.23.5
--------------------------------------------------------------------------------

In [8]:
SIM_CASES = {
    "thai_ben1": {
        "title": "Ben 1",
        "short_title": "Ben1",
        "planet": "ben1",
        "kw_plt": {"color": "C0"},
        "timestep": 1200,
    },
    "thai_ben2": {
        "title": "Ben 2",
        "short_title": "Ben2",
        "planet": "ben2",
        "kw_plt": {"color": "C0"},
        "timestep": 1200,
    },
    "thai_hab1": {
        "title": "Hab 1",
        "short_title": "Hab1",
        "planet": "hab1",
        "kw_plt": {"color": "C0"},
        "timestep": 1200,
    },
    "thai_hab2": {
        "title": "Hab 2",
        "short_title": "Hab2",
        "planet": "hab2",
        "kw_plt": {"color": "C0"},
        "timestep": 1200,
    },
}

MODELS = {
    # "um": {
    #     "model": um,
    #     "title": "UM",
    #     "get_files": load_proc_um_data,
    #     "kw_plt": {"linestyle": "--", "linewidth": 0.75, "dash_capstyle": "round"},
    # },
    "lfric": {
        "model": lfric,
        "title": "LFRic",
        "get_files": lambda sim_label: load_lfric_proc(
            sorted((paths.results_proc_lfric / sim_label).glob(f"{sim_label}*_regr.nc"))
        ),
        "kw_plt": {"linestyle": "-", "linewidth": 1.25},
    },
}

## Load data

In [9]:
runs = {}
for model_key, model_prop in MODELS.items():
    model = model_prop["model"]
    runs[model_key] = {}

    for sim_label, sim_prop in SIM_CASES.items():
        cl = model_prop["get_files"](sim_label)
        const = init_const(sim_prop["planet"], directory=paths.const)
        # calc_derived_cubes(cl, const=const, model=model)

        runs[model_key][sim_label] = AtmoSim(
            cl,
            name=sim_label,
            vert_coord="z",
            planet=sim_prop["planet"],
            const_dir=paths.const,
            model=model,
        )

## Apply diagnostics

In [12]:
time_reduce = partial(last_n_day_mean, days=610, model=lfric)

In [13]:
@const_from_attrs()
@update_metadata(name="greenhouse_effect_parameter", units="K")
def greenhouse_effect(cubelist, kind="all_sky", const=None, model=um):
    r"""
    Calculate the greenhouse effect [K].

    .. math::
        GHE = T_{sfc} - \left(\frac{T_{eff}}{T_{sfc}}\right)^4

    Parameters
    ----------
    cubelist: iris.cube.CubeList
        Input list of cubes.
    kind: str, optional
        Type of GHE:  "all_sky" or "clear_sky"
    model: aeolus.model.Model, optional
        Model class with relevant variable names.

    Returns
    -------
    iris.cube.Cube
        Cube of greenhouse effect parameter.
    """
    t_sfc = cubelist.extract_cube(model.t_sfc)
    if kind == "all_sky":
        toa_olr = cubelist.extract_cube(model.toa_olr)
    elif kind == "clear_sky":
        toa_olr = cubelist.extract_cube(model.toa_olr_cs)
    ghe = t_sfc - (toa_olr / const.stefan_boltzmann) ** 0.25
    return ghe

In [14]:
DIAGS = {
    "t_sfc": {
        "recipe": lambda AS: spatial_mean(time_reduce(AS.t_sfc), model=lfric),
        "fmt": lambda x: f"{x:.1f}",
        "title": r"T\textsubscript{s}",
    },
    "toa_olr": {
        "recipe": lambda AS: spatial_mean(time_reduce(AS.toa_olr), model=lfric),
        "fmt": lambda x: f"{x:.1f}",
        "title": r"TOA OLR",
    },
    # "cre_sw": {
    #     "recipe": lambda AS: spatial_mean(
    #         time_reduce(toa_cloud_radiative_effect(AS._cubes, "sw", model=lfric)),
    #         model=lfric,
    #     ),
    #     "fmt": lambda x: f"{x:.1f}",
    #     "title": r"CRE\textsubscript{SW}",
    # },
    # "cre_lw": {
    #     "recipe": lambda AS: spatial_mean(
    #         time_reduce(toa_cloud_radiative_effect(AS._cubes, "lw", model=lfric)),
    #         model=lfric,
    #     ),
    #     "fmt": lambda x: f"{x:.1f}",
    #     "title": r"CRE\textsubscript{LW}",
    # },
    # "cre": {
    #     "recipe": lambda AS: spatial_mean(
    #         time_reduce(toa_cloud_radiative_effect(AS._cubes, "total", model="lfric")),
    #         model=lfric,
    #     ),
    #     "fmt": lambda x: f"{x:.1f}",
    #     "title": r"CRE",
    # },
    "alb": {
        "recipe": lambda AS: spatial_mean(
            time_reduce(bond_albedo(AS._cubes, model=lfric)), model=lfric
        ),
        "fmt": lambda x: f"{x:.2f}",
        "title": r"$\alpha_\text{p}$",
    },
    "ghe": {
        "recipe": lambda AS: spatial_mean(
            time_reduce(greenhouse_effect(AS._cubes, model=lfric)), model=lfric
        ),
        "fmt": lambda x: f"{x:.1f}",
        "title": r"G",
    },
}

In [26]:
results = {}
for sim_label, sim_prop in SIM_CASES.items():
    dfs = {}
    for model_key, model_prop in MODELS.items():
        model = model_prop["model"]
        _data = {}
        for diag_key, diag_prop in DIAGS.items():
            _data[diag_prop["title"]] = float(
                diag_prop["recipe"](runs[model_key][sim_label]).data
            )
        dfs[model_prop["title"]] = pd.DataFrame(
            _data, index=pd.Index(name="GCM", data=[model_prop["title"]])
        )
    results[sim_label] = pd.concat(dfs, axis="index").droplevel(0).T

In [27]:
for sim_label, sim_prop in SIM_CASES.items():
    df = results[sim_label].T
    formatters = {k: {j["title"]: j["fmt"] for j in DIAGS.values()}[k] for k in df.columns}
    column_format = "l" + "c" * len(DIAGS)
    print(fr"{{}} & \multicolumn{{{len(DIAGS)}}}{{c}}{{{sim_prop['title']}}} \\", end="\r")
    latex_str = df.to_latex(
            formatters=formatters,
            column_format=column_format,
            escape=False,
        )
    latex_str = latex_str.replace(fr"\begin{{tabular}}{{{column_format}}}", "")
    latex_str = latex_str.replace("\end{tabular}\n", "")
    latex_str = latex_str.replace("toprule", "tophline")
    latex_str = latex_str.replace("midrule", "middlehline")
    latex_str = latex_str.replace("bottomrule", "bottomhline")
    print(latex_str)

{} & \multicolumn{4}{c}{Ben 1} \\
\tophline
{} & T\textsubscript{s} & TOA OLR & $\alpha_\text{p}$ &    G \\
GCM   &                    &         &                   &      \\
\middlehline
LFRic &              207.6 &   162.3 &              0.29 & -8.3 \\
\bottomhline

{} & \multicolumn{4}{c}{Ben 2} \\
\tophline
{} & T\textsubscript{s} & TOA OLR & $\alpha_\text{p}$ &   G \\
GCM   &                    &         &                   &     \\
\middlehline
LFRic &              238.8 &   182.6 &              0.19 & 2.3 \\
\bottomhline

{} & \multicolumn{4}{c}{Hab 1} \\
\tophline
{} & T\textsubscript{s} & TOA OLR & $\alpha_\text{p}$ &   G \\
GCM   &                    &         &                   &     \\
\middlehline
LFRic &              242.8 &   182.5 &              0.19 & 5.9 \\
\bottomhline

{} & \multicolumn{4}{c}{Hab 2} \\
\tophline
{} & T\textsubscript{s} & TOA OLR & $\alpha_\text{p}$ &    G \\
GCM   &                    &         &                   &      \\
\middlehline
LFRic &    